images - line by line, cmaera - col by col
    Q: is this hardware specific?
screen waits 1 frame cycle, camera waits 1 frame cycle 
    Q: r they the same?


two kinds of challenges:
    background challenge: one color
    lighting challenge: belt of different colr from background color
        belt is "lighting area"

ROI: region that camera is scanning when the screen is displaying the lighting area

In [1]:
import numpy as np
import cv2 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import pandas as pd
from deepface import DeepFace
import os
import shutil
import random

In [46]:
input_video = "fake_angelina.mp4" # Replace with path of input video
output_directory1 = "output_1"
frames = "videoframes_raw5"
colorchange_csv = 'fake_angelina.csv'

# Video processing

In [47]:
# Split video into frames in the videoframes directory
# Runtime: ~60 seconds

# Delete directory if it already exists and make a new one
if os.path.exists(output_directory1):
   shutil.rmtree(output_directory1)
os.makedirs(output_directory1)

vidcap = cv2.VideoCapture(input_video)
fps = vidcap.get(cv2.CAP_PROP_FPS) # get fps
frametime_list = []
print("FPS", fps)

if vidcap.isOpened():
  while True:
      ret, frame = vidcap.read()
      
      if not ret:
          break
      
      # Get the timestamp of the current frame in milliseconds 
      timestamp = round(vidcap.get(cv2.CAP_PROP_POS_MSEC))
      frametime_list.append(timestamp)
      
      # Save each frame as an image in the output folder
      frame_name = f"frame_{timestamp}.jpg"
      frame_path = os.path.join(output_directory1, frame_name)
      cv2.imwrite(frame_path, frame)

  vidcap.release()

  frametime_list = np.array(frametime_list)

  print(f"Video frames saved in '{output_directory1}'")

FPS 30.00514844688519
Video frames saved in 'output_1'


In [48]:
#CONVERT FRAMES INTO CROPPED FRAMES 
if os.path.exists(frames):
   shutil.rmtree(frames)
os.makedirs(frames)

for filename in sorted(os.listdir(output_directory1)):
    frame_path = os.path.join(output_directory1, filename)
    
    if os.path.isfile(frame_path):  # Check if it's a file (not a subdirectory)
        #face detection and alignment
        img = cv2.imread(frame_path)

        try: #skip frame if can't detect face
            face_objs = DeepFace.extract_faces(img_path = frame_path, 
                    target_size = (224, 224)
            )
        except:
            print("Couldn't detect face in frame " + frame_path)

        face = face_objs[0]['facial_area']
        x,y,w,h = face['x'],face['y'],face['w'],face['h']

        img = img[y:y+h, x:x+w]

    # Save each frame as an image in the output folder
        # frame_name = f"frame_{frame_count:04d}.jpg"
        frame_path = os.path.join(frames, filename)
        cv2.imwrite(frame_path, img)

# Eqn 2

In [36]:
def eqn_2(pixel, color1, color2, E):
    '''
    #INPUTS:
        pixel: a single pixel with all 3 color channels
        color1: background color being shown on screen
        color2: primary color being shown on screen (band)
        E: illuminance for all 3 channels 

    Confirm that I{c1}/I{c2} = E{c1}/E{c2} (where c1 and c2 are the 2 colors being shown on the screen)  
    '''
    iFraction = pixel[color1]/(pixel[color2]+1)
    eFraction = E[color1]/E[color2]
    epsilon = 0.05
    return iFraction- eFraction <= epsilon 

In [9]:
def verifyEqn2(color1, color2, img):
    # Apply Eqn 2 on every pixel between response of lighting challenge and background challenge
    count = 0
    E = [0,0,0]
    E[int(color1)] = 256
    E[int(color2)] = 256
    for r in range(img.shape[0]):
        for c in range(img.shape[1]):
            consistent = eqn_2(img[r][c][:], color1, color2, E)
            if not consistent:
                count+=1
                # print("Not consistent!")
                # return

    return (count/(img.shape[0]*img.shape[1]))

In [10]:
#EQN2 CHECK:
image = cv2.imread('videoframes_raw/frame_0.jpg')
color1 = 1
color2 = 2
E = [0,256,256]
inconsistent = verifyEqn2(color1, color2, image)
print(inconsistent)

0.09526234567901234


# ROI

In [11]:
fps = 30 #(camera)
# 1/120 ms to draw one frame on the screen
def roi(t_u, ct_k, ct_frame, image):
    '''
    INPUTS:
        t_u = time that this color started
        u = top of band
        ct_k = start time to exposure the first column of k-th capture frame
            --> find w/ firstImg
        ct_frame = exposure time of one captured frame 
            --> average time of each frame ? maybe can calculate w/ dict. 
        image = first image whose recording period covers t_u
    '''
    cols = image.shape[1]
    a = cols * (t_u - ct_k)/ct_frame
    b = a+0.2*image.shape[1]
    return [a,b]

In [12]:
def mapColors(color):
    if color == "Red":
        return 0
    if color == "Green":
        return 1
    else:
        return 2
    
def removePercent(s):
    return float("0." + s[:-1])

# colorChanges = pd.read_csv(colorchange_csv)
# colorChanges.iloc[:,0] = colorChanges.iloc[:,0].apply(mapColors)
# colorChanges.iloc[:,1] = colorChanges.iloc[:,1].apply(mapColors)
# colorChanges.iloc[:,2] = colorChanges.iloc[:,2].apply(removePercent)
# colorChanges.iloc[:,3]= colorChanges.iloc[:,3]-colorChanges.iloc[0,3]

# colorChanges.head()

,Background Color,Strip Color,Strip Position,Timestamp
0,2,0,0.1,0
1,2,0,0.3,501
2,2,1,0.6,1001
3,0,2,0.1,1500
4,2,0,0.1,2003


IMPLEMENTATION

In [20]:
#training linear regression models to be used to identify location of band of color
def get_average_vector(roi):
    # Calculate the mean along the height and width (axis 0 and 1), resulting in mean color
    return np.mean(roi, axis=(0, 1))


def lr_fit(rois, targets):
    # predictions = []
    for i in range(len(rois)):
        avg_vector = get_average_vector(rois[i])

        lr = LinearRegression().fit([avg_vector], [targets[i]])
    return lr
    # return predictions

def lr_predict(lr, rois):
    predictions = []
    for i in range(len(rois)):
        avg_vector = get_average_vector(rois[i])
        prediction = lr.predict([avg_vector])
        predictions.append(prediction)
    
    return predictions



In [17]:
# TESTING

def find_closest_filename(folder_path, target):
    closest_filename = None
    closest_distance = float('inf')  # Initialize with a large number
    best_ctk = 0

    # Iterate over each file in the directory
    for filename in os.listdir(folder_path):
        if filename.startswith('frame_') and filename.endswith('.jpg'):
            # Extract the number from the filename
            number_part = filename.replace('frame_', '').replace('.jpg', '')
            try:
                number = int(number_part)
                if number <= target: # tu - ctk >= 30
                # if target - number < 24:
                    # Calculate the absolute difference from the target
                    distance = abs(number - target)

                    # Update the closest filename if this file is closer
                    if distance < closest_distance:
                        closest_distance = distance
                        closest_filename = filename
                        best_ctk = number
            except ValueError:
                # Handle the case where conversion to int fails
                continue

    return closest_filename, best_ctk


rois = []
targets = []

data_files = ['colors.csv', 'shun.csv', 'shreya.csv', 'michelle.csv']
data_frames = ['videoframes_raw', 'videoframes_raw2', 'videoframes_raw3', 'videoframes_raw4']

for j in range(len(data_files)):
    colorChanges = pd.read_csv(data_files[j])
    colorChanges.iloc[:,0] = colorChanges.iloc[:,0].apply(mapColors)
    colorChanges.iloc[:,1] = colorChanges.iloc[:,1].apply(mapColors)
    colorChanges.iloc[:,2] = colorChanges.iloc[:,2].apply(removePercent)
    colorChanges.iloc[:,3]= colorChanges.iloc[:,3]-colorChanges.iloc[0,3]

    for i in range(colorChanges.shape[0] - 1): #for each color change -> TODO: this should b number of lines in csv
        color1, color2 = colorChanges.iloc[i,0],colorChanges.iloc[i,1]
        u, startTime = colorChanges.iloc[i,2], colorChanges.iloc[i,3]
        offset = random.uniform(0,1) * 3 # randomly select a ms offset 
        t_u = startTime+offset
        
        # find closest filepath based on timestamp
        filename, ct_k = find_closest_filename(data_frames[j], t_u)

        image = cv2.imread(f'{data_frames[j]}/{filename}')
        
        # ct_k = int(filename.replace('frame_', '').replace('.jpg', '')) # timestamp
        ct_frame = 30

        if t_u - ct_k < 24:
            my_roi = roi(t_u, ct_k, ct_frame, image)

            roi_image = image[:, int(my_roi[0]):int(my_roi[1])]

            u = roi_image.shape[0] * colorChanges.iloc[i,2]  # redefine u?
            target = u + 0.1 # u + d / 2

            rois.append(roi_image)
            targets.append(target)

In [22]:
train_x, test_x, train_y, test_y = train_test_split(rois, targets, test_size=0.33)
lr_model = lr_fit(train_x, train_y)
predictions = lr_predict(lr_model, test_x)

score = mse(test_y, predictions)
print(predictions)
print("SCORE", score)

[array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([126.7]), array([

In [49]:
frames = 'videoframes_raw5'
datafile = 'fake_angelina.csv'
colorChanges = pd.read_csv(datafile)
colorChanges.iloc[:,0] = colorChanges.iloc[:,0].apply(mapColors)
colorChanges.iloc[:,1] = colorChanges.iloc[:,1].apply(mapColors)
colorChanges.iloc[:,2] = colorChanges.iloc[:,2].apply(removePercent)
colorChanges.iloc[:,3]= colorChanges.iloc[:,3]-colorChanges.iloc[0,3]

In [50]:
dVals = []

for i in range(len(datafile)): #for each color change
    color1, color2 = colorChanges.iloc[i,0],colorChanges.iloc[i,1]
    u, startTime = colorChanges.iloc[i,2], colorChanges.iloc[i,3]
    offset = random.randint(2,6)# randomly select a ms offset 
    t_u = startTime+offset

    filename, ct_k = find_closest_filename(frames, t_u)
    img = cv2.imread(f'{frames}/{filename}')

    
    # imgIndex = np.sum(frametime_list<t_u)-1 #find all frames capturing time less than t_u, select last one
    # imgPath = f'{frames}/frame_' + str(frametime_list[imgIndex]) + ".jpg"
    # print(imgPath)
    # img = cv2.imread(imgPath)
    # print(img.shape)

    exposureTime = 50 # ideally should get file timestamp, if we get frametime list here for the current video file, can get it from there
    #(AFTER ct_k) - ct_k
    #frametime_list[imgIndex+1]-frametime_list[imgIndex]

    #calculate eqn2 
    inconsistency = verifyEqn2(color1, color2, img)
    print("inconsistency: ", inconsistency)
    if inconsistency >0.1:     #STOP HERE IF DOESNT PASS
        print("INCONSISTENT")
        break

    a,b = roi(t_u, startTime, exposureTime, img)
    a = round(a)
    b = round(b)
    print("a, b: ", a,b)

    imgRows = img.shape[0] 
    roi_image = img[:, int(a):int(b)]

    rois = [roi_image]
    y_hat_i =lr_predict(lr_model, rois)
    d_i = y_hat_i[0] - (u + u+imgRows*0.2)/2 #band is shown on 20% of screen
    dVals.append(d_i)


mean = np.mean(dVals)
var = np.var(dVals)
threshold = -5

print(mean * np.sqrt(var) < np.exp(threshold))

inconsistency:  0.8984210526315789
INCONSISTENT
False
